In [1]:
from yahpo_train.learner  import *
from yahpo_train.metrics import *
from yahpo_train.cont_scalers import *
from yahpo_gym.benchmarks import lcbench, rbv2, nb301, fcnet, taskset, fcnet
from yahpo_gym.configuration import cfg
from fastai.callback.wandb import *
from functools import partial

## Training a surrogate:

We provide a function `fit_config` that allows for training a surrogate with a set of hyperparameters and the option to export the surrogate (this can overwrite existing surrogates!).

A particularity is that we use a set of so called `ContTransformers` in order to transfer continuous variables to a scale better suited for optimization! 
This has a strong effect on the resulting performance and should therefore be optimized!

In [2]:
def fit_config(key, embds_dbl=None, embds_tgt=None, tfms=None, lr = 1e-4, epochs=25, frac=0.1, bs=2048, export=False):
    """
    Fit function with hyperparameters
    """
    cc = cfg(key)
    dls = dl_from_config(cc, bs=bs, frac=frac)

    # Construct embds from transforms. tfms overwrites emdbs_dbl, embds_tgt
    if tfms is not None:
        embds_dbl = [tfms.get(name) if tfms.get(name) is not None else ContTransformerNone for name, cont in dls.all_cols[dls.cont_names].iteritems()]
        embds_tgt = [tfms.get(name) if tfms.get(name) is not None else ContTransformerNone for name, cont in dls.ys.iteritems()]

    # Instantiate learner
    f = ResNet(dls, embds_dbl=embds_dbl, embds_tgt=embds_tgt)
    l = SurrogateTabularLearner(dls, f, loss_func=nn.MSELoss(reduction='mean'), metrics=nn.MSELoss)
    l.metrics = [AvgTfedMetric(mae),  AvgTfedMetric(r2), AvgTfedMetric(spearman)]
    l.add_cb(MixHandler)
    l.add_cb(EarlyStoppingCallback(patience=3))

    # Fit
    l.fit_flat_cos(epochs, lr)

    if export:
        l.export_onnx(cc)


## Example 1: NasBench 301
Find an example for training the `NASBENCH 301` surrogate below:

We supply a list of `ContTransformer`'s to our `fit_config` function that define the specific transformers that should be applied for this scenario:


In [ ]:

def fit_nb301(key = 'nb301', **kwargs):
    embds_dbl = [partial(ContTransformerMultScalar, m = 1/52)]
    embds_tgt = [partial(ContTransformerMultScalar, m = 1/100), ContTransformerRange]
    fit_config(key, embds_dbl=embds_dbl, embds_tgt=embds_tgt, **kwargs)


## Example 2: rbv2_super

A more involved example is the `rbv2_super` surrogate, where multiple different transformers are used depending on the input and output variables.


In [ ]:

def fit_rbv2_super(key = 'rbv2_super', **kwargs):
    # Transforms
    tfms = {}
    [tfms.update({k:ContTransformerRange}) for k in ["mmce", "f1", "auc", "aknn.k", "aknn.M", "rpart.maxdepth", "rpart.minsplit", "rpart.minbucket", "xgboost.max_depth"]]
    [tfms.update({k:partial(ContTransformerLogRange)}) for k in ["timetrain", "timepredict", "svm.cost", "svm.gamma"]]
    [tfms.update({k:partial(ContTransformerLogRange, logfun=torch.log2,  expfun=torch.exp2 )}) for k in ["glmnet.s", "rpart.cp", "aknn.ef", "aknn.ef_construction", "xgboost.nrounds", "xgboost.eta", "xgboost.gamma", "xgboost.lambda", "xgboost.alpha", "xgboost.min_child_weight", "ranger.num.trees", "ranger.min.node.size", 'ranger.num.random.splits']]
    [tfms.update({k:ContTransformerNegExpRange}) for k in ["logloss"]]

    fit_config(key, tfms=tfms, **kwargs)


In [32]:
cc = cfg('fcnet')

b =BenchmarkSet('fcnet')

b.config_space.get_hyperparameter_names() == cc.hp_names

False